<a href="https://colab.research.google.com/github/jacobfvanzyl/KML_Extractor/blob/master/KML_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XSIT KML Validator

## How-to:
1. Place KML file in working directory
1. 

# Environment setup

In [1]:
# Only run this cell on every new instance
!pip install fastkml
!pip install xmltodict
!pip install pprint

     |████████████████████████████████| 71kB 4.4MB/s 
  Created wheel for fastkml: filename=fastkml-0.11-cp36-none-any.whl size=61866 sha256=b630c92b5838540c48b927992e0ca963c8c2d43dce746f23e39af865ecb005d2
  Stored in directory: /root/.cache/pip/wheels/55/01/ea/6191eb73e0894743d02b33a2b1a570e85242844d810804fbf2
  Created wheel for pygeoif: filename=pygeoif-0.7-cp36-none-any.whl size=19248 sha256=0dc99a88bac46958ba06e636e61dbdc6cd7a7607db5fb0afa55c13363281e2d3
  Stored in directory: /root/.cache/pip/wheels/60/6e/a7/3d3eef59ac84a86663d0f5c5a92091f5056e9aeb6588c4de34
Successfully built fastkml pygeoif
  Created wheel for pprint: filename=pprint-0.1-cp36-none-any.whl size=1251 sha256=9e7408fe2b206a2a21d4560518dab74abe99870f2d2cf9ea2d04dd849f4a25c7
  Stored in directory: /root/.cache/pip/wheels/42/d4/c6/16a6495aecc1bda5d5857bd036efd50617789ba9bea4a05124
Successfully built pprint


# Imports

In [0]:
import os
from fastkml import kml
import xmltodict
import json
import pprint
from shapely.geometry import Point, Polygon

debug = True

# Load data

In [3]:

if debug: 
    batch_path = "/content/drive/My Drive/KeyPhase/Development/PointInPolygon/"
batch_name = "test"
kml_path = batch_path + "input/" + batch_name + ".kml"
json_path = "/content/drive/My Drive/KeyPhase/Development/PointInPolygon/input/" + batch_name + ".json"
debug_path = batch_path + 'output/debug/'

os.makedirs(os.path.dirname(debug_path), exist_ok=True)

# Write XML to JSON
with open(kml_path) as xml_file:
    with open(json_path, 'w') as json_file:
        json.dump(xmltodict.parse(xml_file.read()), json_file)

# Traverse down into relevant level
data = ""
with open(json_path) as json_data:
    contents = json_data.read()
    data = json.loads(contents)
    data = data["kml"]["Document"]

print("Successfully created JSON file and loaded file: " + data['name'] + " into memory")

data = data['Folder']['Folder']
print(data)
if debug:
    with open(debug_path + 'data.json', 'w') as json_file:
        json.dump(data, json_file)

FileNotFoundError: ignored

# Nested data extraction

In [0]:

organisations = []
farms = []
blocks = []
traps = []

for org in data:
    # Orgs:
    if debug:
        print("- O: " + org['name'])
    organisations.append(org['name'])

    # Farms:
    for farm in org["Folder"]:
        if debug:
            print("-- F: " + farm['name'])
        farms.append(farm['name'])

        # Blocks:
        if 'Placemark' in farm:
            for marker in farm['Placemark']:
                if 'Polygon' in marker:
                    if debug:
                        print("--- B: " + marker['name'])
                elif 'Point' in marker:
                    if debug:
                        print("--- T: " + marker['name'])
        else:
            print("!- F: " + farm['name'] + " is empty!")
            
                
        

print("Orgs: " + str(organisations))
print("Farms: " + str(farms))
print("Blocks: " + str(blocks))
print("Traps: " + str(traps))


# Validation

In [0]:
# Basic Validation
def checkDuplicatesInList(List):
    # Check list for duplicates
    if len(List) == len(set(List)):
        return False
    else:
        return True

# Orgs:
duplicate_orgs = checkDuplicatesInList(organisations)
if duplicate_orgs:
    print("Duplicate ORGANISATIONS found. Please merge Organisation folders in KML file.")
else:
    print("No duplicate Organisations found.")

# Farms:
duplicate_farms = checkDuplicatesInList(farms)
if duplicate_farms:
    print("Duplicate FARMS found. Please check Farm folders in KML file.")
else:
    print("No duplicate Farms found.")